# Web Scraping Homework - Mission to Mars

In [3]:
#import dependencies

from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

In [4]:
# Setup config variables to enable Splinter interaction with browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

NameError: name 'ChromeDriverManager' is not defined

In [ ]:
# Visit Nasa news url through splinter module
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [ ]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

## NASA Mars News

In [ ]:

# Get the latest News Title and Paragraph Text
news_title = soup.find('li', class_='slide').find('div', class_='content_title').text
news_p = soup.find('li', class_='slide').find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)